# PR 236 - Add weighted time averages with time dimension removed

Floating point closeness validation with CDAT/cdutil

Link: https://github.com/xCDAT/xcdat/pull/236

In [1]:
import cdms2
import cdutil
import numpy as np
import xarray as xr

import xcdat

In [2]:
filepath = "/home/vo13/XCDAT/xcdat/input/demo_data/CMIP5_demo_data/pr_Amon_ACCESS1-0_historical_r1i1p1_185001-200512.nc"

ds = xcdat.open_dataset(filepath)
ds_cdat = cdms2.openDataset(filepath)

## Calculate averages weighted month
- cdutil calculates weights based on the original frequency (I think)
- xcdat has an argument for the weight frequency
- I also calculate averages using native xarray (which is what xcdat wraps)

In [3]:
# cdutil
pr_avg_cdat = cdutil.averager(ds_cdat["pr"], axis="t")

# xcdat
pr_avg_xcdat = ds.temporal.average("pr", freq="month")["pr"]

# xarray
month_length = ds.time.dt.days_in_month
weights = (
    month_length.groupby("time.month") / month_length.groupby("time.month").sum()
)
pr_avg_xarray = ds.pr.weighted(weights).mean(dim="time")

### xCDAT vs. xarray

In [ ]:
np.testing.assert_allclose(pr_avg_xcdat.values, pr_avg_xarray.values)

There no difference between xCDAT and xarray. This is expected because xCDAT simply wraps those xarray calls in a convenience method (`ds.temporal.average()`).

### CDAT vs. xCDAT and xarray

In [4]:
np.testing.assert_allclose(pr_avg_cdat.data, pr_avg_xcdat.values)

AssertionError: 
Not equal to tolerance rtol=1e-07, atol=0

Mismatched elements: 27840 / 27840 (100%)
Max absolute difference: 1.25452385e-06
Max relative difference: 0.02367522
 x: array([[1.771111e-06, 1.771111e-06, 1.771111e-06, ..., 1.771111e-06,
        1.771111e-06, 1.771111e-06],
       [1.885413e-06, 1.874696e-06, 1.863779e-06, ..., 1.921356e-06,...
 y: array([[1.773081e-06, 1.773081e-06, 1.773081e-06, ..., 1.773081e-06,
        1.773081e-06, 1.773081e-06],
       [1.886394e-06, 1.875624e-06, 1.864662e-06, ..., 1.922490e-06,...

In [ ]:
np.testing.assert_allclose(pr_avg_cdat.data, pr_avg_xarray.values)

There is a max relative difference of about 0.0237, or 2.37% with CDAT and xCDAT/xarray.

This could be due to a multitude of reasons, including floating point rounding as a result of the grouping.